Trying to replicate results from https://www.researchgate.net/profile/Salma-Alhagry/publication/320802497_Emotion_Recognition_based_on_EEG_using_LSTM_Recurrent_Neural_Network/links/5b39ff5a0f7e9b0df5e4cf26/Emotion-Recognition-based-on-EEG-using-LSTM-Recurrent-Neural-Network.pdf



In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.io
import scipy.signal
from einops import reduce, rearrange
import numpy as np

In [2]:
# Constants used to define data paths
ROOT_DIR = './'
RAW_DIR = 'data/matlabPREPROCESSED'
PROCESSED_DIR = 'data/graphProcessedData'

In [3]:
raw_file_names = os.listdir(RAW_DIR)
raw_file_names.sort()
all_subjects = [scipy.io.loadmat(f'{RAW_DIR}/{name}') for name in raw_file_names]

In [4]:
all_data = [subject['data'] for subject in all_subjects]
all_labels = [subject['labels'] for subject in all_subjects]
labels = np.stack(all_labels)
data = np.stack(all_data)

In [5]:
data = data[:,:,:32,128*3:]

In [6]:
all_labels = reduce(labels,'s v c -> (s v) c','mean')
all_data = reduce(data,'s v c d -> (s v) c d','mean')

In [7]:
data.shape, all_data.shape, labels.shape, all_labels.shape

((32, 40, 32, 7680), (1280, 32, 7680), (32, 40, 4), (1280, 4))

In [8]:
participant = 2
participant_data = data[participant][:,:,128*3:]

In [9]:
data.shape

(32, 40, 32, 7680)

In [10]:
processed = []
for participant_data in data:
    for video in participant_data:
        video = video.reshape(32,-1,128)
        processed.append(video)
#         processed.append(scipy.signal.periodogram(video)[1])

In [11]:
all_data = torch.FloatTensor(processed)
# all_labels = labels[participant]
all_data.shape, all_labels.shape

(torch.Size([1280, 32, 60, 128]), (1280, 4))

In [12]:
train_test_split = 0.75
target = 0 # valence
split_idx = int(all_labels.shape[0]*train_test_split)

In [13]:
train_x, train_y = all_data[:split_idx], torch.FloatTensor(all_labels[:split_idx,target])
test_x, test_y = all_data[split_idx:], torch.FloatTensor(all_labels[split_idx:,target])
train_y = (train_y > 5).float()
test_y = (test_y > 5).float()

In [14]:
val_train_split = 0.9
split_idx = int(train_y.shape[0]*val_train_split)
val_x, val_y = train_x[split_idx:], train_y[split_idx:]
train_x, train_y = train_x[:split_idx], train_y[:split_idx]

In [15]:
train_x.shape, val_x.shape, test_x.shape, train_y.shape

(torch.Size([864, 32, 60, 128]),
 torch.Size([96, 32, 60, 128]),
 torch.Size([320, 32, 60, 128]),
 torch.Size([864]))

In [16]:
train_data = [(x,y) for x,y in zip(train_x,train_y)]
val_data = [(x,y) for x,y in zip(val_x,val_y)]
test_data = [(x,y) for x,y in zip(test_x,test_y)]

In [17]:
BS = 64
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BS, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=BS)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BS)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [43]:
class Model(torch.nn.Module):
    def __init__(self, in_channels,hidden_channels):
        super(Model, self).__init__()
        self.lstms = nn.ModuleList([nn.LSTM(in_channels, hidden_channels, 4, dropout=0.05, batch_first=True).to(device) for i in range(32)])
        self.lin1 = nn.Linear(hidden_channels*32,hidden_channels).to(device)
        self.lin2 = nn.Linear(hidden_channels,1).to(device)
        
    def forward(self, x):
        bs = x.shape[0]
        hs = []
        for i,lstm in enumerate(self.lstms):
            o, (h,c) = lstm(x[:,i,:,:])
            hs.append(h[-1])
            
        hs = torch.stack(hs)
        x = hs.reshape(bs,-1)

        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        return x.view(-1).sigmoid()

In [44]:
model = Model(all_data.shape[-1],hidden_channels=8)     
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f'Model parameter count: {pytorch_total_params}')

# optimizer = torch.optim.Adadelta(model.parameters(), lr=.1, rho=0.9, eps=1e-06, weight_decay=1e-5)
optimizer = torch.optim.SGD(model.parameters(),lr=1e-1, weight_decay=1e-3)
# optimizer = torch.optim.Adam(model.parameters(), lr=.05, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=True)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01, weight_decay=1e-6)

criterion = nn.BCELoss()

model = model.to()

def train():
    model.train()
    losses = []
    right = 0
    tot = 0
    for x,y in train_loader:
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out,y)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        right += torch.eq(out > 0.5, y > 0.5).sum().item()
        tot += y.shape[0]
    return np.array(losses).mean(), right/tot

def test(loader,verbose=False):
    model.eval()
    losses = []
    right = 0
    tot = 0
    for x,y in loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        if verbose:
            print(out,y)
        right += torch.eq(out > 0.5, y > 0.5).sum().item()
        tot += y.shape[0]
        loss = criterion(out,y)
        losses.append(loss.item())
    return np.array(losses).mean(), right/tot

best_val_loss = np.inf
esp = 0
MAX_ESP = 50
for epoch in range(1, 1000):
    train_loss, train_acc = train()
    val_loss, val_acc = test(val_loader)
    if val_loss <= best_val_loss:
        best_val_loss = val_loss
        esp = 0
    else:
        esp += 1
        if esp >= MAX_ESP:
            break
#     if epoch%5 == 0:
    print(f'Epoch {epoch} ;t loss: {train_loss:.5f} ;t acc: {train_acc:.2f} ;v loss: {val_loss:.5f} ;v acc: {val_acc:.2f}')

print('Finished training')

Model parameter count: 198673
Epoch 1 ;t loss: 0.69544 ;t acc: 0.46 ;v loss: 0.69350 ;v acc: 0.46
Epoch 2 ;t loss: 0.69242 ;t acc: 0.53 ;v loss: 0.69807 ;v acc: 0.47
Epoch 3 ;t loss: 0.69182 ;t acc: 0.54 ;v loss: 0.70019 ;v acc: 0.48
Epoch 4 ;t loss: 0.69090 ;t acc: 0.54 ;v loss: 0.70182 ;v acc: 0.48
Epoch 5 ;t loss: 0.68949 ;t acc: 0.54 ;v loss: 0.70265 ;v acc: 0.48
Epoch 6 ;t loss: 0.68883 ;t acc: 0.54 ;v loss: 0.70483 ;v acc: 0.48
Epoch 7 ;t loss: 0.68911 ;t acc: 0.54 ;v loss: 0.70610 ;v acc: 0.48
Epoch 8 ;t loss: 0.68975 ;t acc: 0.54 ;v loss: 0.70613 ;v acc: 0.48
Epoch 9 ;t loss: 0.69104 ;t acc: 0.54 ;v loss: 0.70527 ;v acc: 0.48
Epoch 10 ;t loss: 0.69046 ;t acc: 0.54 ;v loss: 0.70436 ;v acc: 0.48
Epoch 11 ;t loss: 0.69029 ;t acc: 0.54 ;v loss: 0.70378 ;v acc: 0.48
Epoch 12 ;t loss: 0.68959 ;t acc: 0.54 ;v loss: 0.70503 ;v acc: 0.48
Epoch 13 ;t loss: 0.69013 ;t acc: 0.54 ;v loss: 0.70362 ;v acc: 0.48
Epoch 14 ;t loss: 0.68934 ;t acc: 0.54 ;v loss: 0.70428 ;v acc: 0.48
Epoch 15 ;t l

In [ ]:
loss, acc = test(test_loader)
print(f'Test loss: {loss} ; Test acc: {acc}')

# TODO: esp, kfold val , scheduler(?)